In [60]:
from math import tanh
import numpy as np
from numpy import array
from scipy import optimize
#Box constraints in 3 methods : 1- projected gradient descent(clipps the coordinates to be inside the box)
                               #2- the xi are not clipped , but instead we clipp the gradient: we replace f(x+delta) by f(min(max(x+delta,0),1))
                               #3-but the best is the one below with the variable_change tanh

# -*- coding: utf-8 -*-
"""Copie de Projet SD - Partie 1.ipynb
Automatically generated by Colaboratory.
Original file is located at
    https://colab.research.google.com/drive/1m4u7cVrOUqmZNV0ZjOPgj5y9yeYvqI_s
"""

from __future__ import print_function
import torch
# Packages to load CIFAR10
import torchvision
import torchvision.transforms as transforms
# Packages to define Neural Network
import torch.nn as nn
import torch.nn.functional as F
# Package to define an optmizer
import torch.optim as optim
#import tensorflow as tf




import numpy as np

delta=[0,0,0]
k=0.09

# correct solution:
def softmax(x,i):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    output=e_x / e_x.sum(axis=0) # only difference
    return output[i]


#Define a CNN
'''
A neural network is a function F(x)=y that accepts an input
x belonging to Rn and produces an output y belonging to Rm,
y is a probability distribution that x has class i
'''


def norm_p(delta, p):
    "First-pass implementation of p-norm."
    return (np.abs(delta)**p).sum() ** (1./p)


def variable_change(w,x,i):#ensures that x+delta belongs to [0,1]

  delta=0.5*(tanh(w[i])+1)-x[i] #this is equals to delta
  return delta
  #since -1<=tanh(w)<=1, it follows 0<=x_i+delta_i<=1, so this solution will automatically be valid, we can think of this method as,a clipped descent gradient that eliminates 
  #problem of getting stuck in extreme regions


def objectif_function(my_array,i,t):#this is f

  if(i!=t):
    return max(softmax(my_array,i)-softmax(my_array,t),0.12)
  else:
    return k

  
#what we're trying to do is to minimize D(x,x+delta)
  #such that f(x+delta)<=0 and x+delta belongs to [0,1]
  

def function_to_minimize(delta,p,x,c,i,t):
  return norm_p(delta,p)+c*objectif_function(x+delta,i,t)


def attack_l2(w,x,c,t):
  for i in range(len(x)):
    delta[i]=variable_change(w,x,i)
    w[i]=function_to_minimize(delta[i],2,x,c,i,t)
  return w[i]


  
x0=np.array([0.04,0.1,1.5])
w0=np.array([0.05,12,0.8])
c=0.1
result = optimize.minimize(attack_l2,x0,args=(w0,c,1),method='SLSQP',
                           options={'maxiter':100})


print(result)

print(np.array(delta)+result['x'])

     fun: 0.812
     jac: array([16173143.12556883, 16173143.12556883, 16173142.62668798])
 message: 'Optimization terminated successfully.'
    nfev: 61
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 0.062, 12.009,  0.812])
[0.012 0.009 0.012]
